# Train model using tensorflow embed data 

In [1]:
# import lib
import numpy as np
import json
import pandas as pd
import os
from sklearn import svm

In [2]:
df = pd.read_csv("DCForTrain.csv")
df.head()

,sentences,embed,topic
0,Congressman Carol Miller ?,"[0.05492091551423073, 0.023301545530557632, 0....",healthcare
1,"Dear Friend, Happy Presidents' Day weekend!","[-0.03798495605587959, -0.06996205449104309, 0...",healthcare
2,On this day we honor all that our Presidents h...,"[-0.06237169727683067, -0.08522521704435349, 0...",healthcare
3,This week was filled with many exciting develo...,"[-0.04813029244542122, -0.04647025093436241, -...",healthcare
4,I introduced legislation to fill the remaining...,"[-0.04485123232007027, -0.07864920049905777, -...",healthcare


In [3]:
path = 'cleanedDCinboxResource/'
dest_path = 'encodedDCinboxResource/'
dirs = os.listdir(path)
dirs
topics = map(lambda x: x.replace('.json.csv',''), dirs) 
topics = list(topics)
topics_df = pd.DataFrame(topics)
topics_index = list(topics_df.index)
topics_index

topic_dict = dict(zip(topics, topics_index))
topic_dict

{'healthcare': 0,
 'pet': 1,
 'business': 2,
 'fashion beauty': 3,
 'fitness sports': 4,
 'food beverage': 5,
 'media enterainment': 6,
 'lifecycle home': 7,
 'consumer': 8,
 'green cleantech': 9,
 'children education': 10,
 'software tech': 11}

In [4]:
# Set up training data
df_train = pd.DataFrame()

for i in dirs:
    temp = pd.read_csv("encodedDCinboxResource/" + i)
    temp['topic'] = i.replace('.json.csv','')
    df_train = df_train.append(temp.iloc[:temp.index.size//2,:])


    
# Set up for X_train
X_train = df_train[['embed']]
X_train['embed'] = X_train['embed'].apply(json.loads)
# X_train['embed']
X_train = pd.DataFrame(X_train['embed'].tolist(), columns=range(0, 512))
X_train.head(2)

# X_train = df.copy(deep=True) # copy the dataframe 
# X_train = X_train[['embed']]
# X_train = X_train.iloc[:22981,:]
# X_train['embed'] = X_train['embed'].apply(json.loads)
# # X_train['embed']
# X_train = pd.DataFrame(X_train['embed'].tolist(), columns=range(0, 512))
# X_train.head(2)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,0.054921,0.023302,0.020982,-0.026948,-0.023186,-0.065200,-0.058983,-0.035443,0.048876,-0.046086,...,-0.006283,-0.028407,0.003497,-0.021077,-0.052233,-0.006062,-0.033835,0.056937,0.031446,0.074664
1,-0.037985,-0.069962,0.037102,0.048221,0.053405,0.058545,-0.050000,-0.013386,-0.034736,-0.045103,...,0.001811,0.020552,0.066109,0.010746,-0.039018,-0.012361,0.022109,-0.010583,-0.054189,0.020036


In [5]:
# Set up for y_train

y_train = df_train.copy(deep=True)
y_train = y_train[['topic']]
y_train = y_train.iloc[:22981,:]
y_train = y_train['topic'].map(topic_dict)
y_train.head(2)

0    0
1    0
Name: topic, dtype: int64

In [8]:
# settinf up model

clf = svm.SVR()
clf.fit(X = X_train, y = y_train)

# dump the model for further usage
from joblib import dump, load
dump(clf, 'tweetModel.joblib') 

In [ ]:
Y_predict = clf.predict(X_train)
output=pd.DataFrame(data={"y_train": y_train, "ibu":Y_predict}) 

output.to_csv(path_or_buf="resultssvm.csv",index=False,quoting=3,sep=',')

In [11]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline

# pipeline and fit to training set
pipeline = make_pipeline(
    StandardScaler(),
    SGDRegressor()
)
pipeline.fit(X=X_train, y=y_train)
    
scores = cross_val_score(pipeline, 
                         X=X_train,
                         y=y_train,
                         cv=5)  # cv refers to number of cross-validation cuts
scores

array([-68.62705433, -23.26613862,  -2.7808134 , -24.05696711,
       -64.09164676])

In [14]:
Y_predict = pipeline.predict(X=X_train)
output=pd.DataFrame(data={"y_train": y_train, "ibu":Y_predict}) 

output.to_csv(path_or_buf="results.csv",index=False,quoting=3,sep=',')